In [18]:
import pandas as pd
import numpy as np
import datetime
import math
import sklearn.cross_validation
from scipy import stats

In [19]:
train = pd.read_csv('data_original/train.csv')
test = pd.read_csv('data_original/test.csv')

print len(train.Point.unique())
print len(test.Point.unique())
a = set(train.Point.unique()) - (set(train.Point.unique()) & set(test.Point.unique()))
for i in a:
    print i
    train = train[train.Point != i]

    
test['Count'] = None
frame = [train, test]
common = pd.concat(frame)
common.Date = common.Date.apply(lambda x: pd.to_datetime(x, format='%d.%m.%Y'))
common = common.sort_values(by=['Date'])

114
114


In [20]:
common.to_csv('data_original/common.csv', index=None)
train.to_csv('data_original/train.csv', index=None)
test.to_csv('data_original/test.csv', index=None)

# date.csv

In [21]:
date = pd.read_csv('data_original/common.csv')[['Date']]
date.Date = date.Date.apply(lambda x: pd.to_datetime(x))

In [22]:
start_date = date.iloc[0]['Date']
date['Year'] = date.Date.apply(lambda x: x.year)
date['Month'] = date.Date.apply(lambda x: x.month)
date['MonthDay'] = date.Date.apply(lambda x: x.day)
date['WeekDay'] = date.Date.apply(lambda x: x.weekday())
date['Point'] = pd.Series(common.Point.as_matrix())

In [23]:
date.shape

(53385, 6)

In [24]:
common.shape

(53385, 3)

In [25]:
date['YearDay'] = date.Date.apply(lambda x: (x-pd.to_datetime(str(x.year)+'-01-01')).days)

In [26]:
date['ThroughMonth'] = date.Date.apply(lambda x: (x.year - start_date.year) * 12 + x.month - start_date.month)

In [27]:
date['ThroughWeek'] = date.Date.apply(lambda x: int( ( (x - start_date).days + start_date.weekday()) / 7))
#теперь нет смещения, это могло вводить неоднозначноть, т.е. одной недели могли противоречить две фичи

In [28]:
date['ThroughDay'] = date.Date.apply(lambda x: (x - start_date).days)

In [29]:
date.to_csv('data_transform/date.csv', index=None)

In [30]:
date.head(25)

,Date,Year,Month,MonthDay,WeekDay,Point,YearDay,ThroughMonth,ThroughWeek,ThroughDay
0,2011-10-06,2011,10,6,3,Сыктывкар-1,278,0,0,0
1,2011-10-07,2011,10,7,4,Сыктывкар-1,279,0,0,1
2,2011-10-09,2011,10,9,6,Сыктывкар-1,281,0,0,3
3,2011-11-15,2011,11,15,1,Сыктывкар-1,318,1,6,40
4,2011-12-08,2011,12,8,3,Сыктывкар-1,341,2,9,63
5,2011-12-09,2011,12,9,4,Сыктывкар-1,342,2,9,64
6,2011-12-10,2011,12,10,5,Сыктывкар-1,343,2,9,65
7,2011-12-11,2011,12,11,6,Сыктывкар-1,344,2,9,66
8,2011-12-13,2011,12,13,1,Сыктывкар-1,346,2,10,68
9,2011-12-14,2011,12,14,2,Сыктывкар-1,347,2,10,69


# target.csv

In [31]:
sizeValidationSample = 0.15

In [32]:
target = pd.read_csv('data_original/common.csv')[['Count']]
#target['Point'] = pd.Series(common.Point.as_matrix())
target['IsTest'] = target.Count
target['IsTrain'] = pd.Series(range(target.shape[0]))
target['IsValidation'] = pd.Series(range(target.shape[0]))

In [33]:
target['IsTest'] = target.IsTest.apply(lambda x: 1 if math.isnan(x) else 0)

In [34]:
withCountValue = target[target.IsTest == 0].index
indexTrain, indexValidation = sklearn.cross_validation.train_test_split(withCountValue, test_size=sizeValidationSample)

In [35]:
target['IsTrain'] = target.IsTrain.apply(lambda x: 1 if ((x in indexTrain) & (target.IsTest[x] == 0)) else 0)
target['IsValidation'] = target.IsValidation.apply(lambda x: 1 if ((x in indexValidation) & (target.IsTest[x] == 0)) else 0)

In [36]:
target.to_csv('data_transform/target.csv', index = None)

In [37]:
target.head(15)

,Count,IsTest,IsTrain,IsValidation
0,8.0,0,1,0
1,88.0,0,1,0
2,1.0,0,1,0
3,1.0,0,1,0
4,2.0,0,1,0
5,43.0,0,1,0
6,10.0,0,1,0
7,1.0,0,1,0
8,60.0,0,1,0
9,NaN,1,0,0


# city.csv


In [38]:
cities = pd.read_csv('data_original/common.csv')[['Point']].Point.unique()
city = pd.DataFrame()
city['Point'] = pd.Series(cities)
city['CityName'] = city.Point.apply(lambda x: '-'.join(x.split('-')[0:-1]))
city_names = city.CityName.unique()
city['CityID'] = city.CityName.apply(lambda x: np.where(city_names == x)[0][0]) 
city['BranchNumber'] = city.Point.apply(lambda x: x.split('-')[-1:][0])
# Москва 1 -> Москва
city['CityName'] = city.CityName.apply(lambda x: ('Москва' if x == 'Москва 1' else x))
city.to_csv('data_transform/city.csv', index = None)

In [39]:
'''
# geo
city = pd.read_csv('data_transform/city.csv')
cities = city.CityName.unique()
geo = pd.read_csv('data_transform/geo.csv', names=['CityName', 'lat', 'lng'])
geo.CityName = pd.Series(cities)
city = pd.merge(city, geo, how='left', on='CityName')
city.to_csv('data_transform/city.csv', index=None)
city.head(10)
'''

"\n# geo\ncity = pd.read_csv('data_transform/city.csv')\ncities = city.CityName.unique()\ngeo = pd.read_csv('data_transform/geo.csv', names=['CityName', 'lat', 'lng'])\ngeo.CityName = pd.Series(cities)\ncity = pd.merge(city, geo, how='left', on='CityName')\ncity.to_csv('data_transform/city.csv', index=None)\ncity.head(10)\n"

In [40]:
'''
# get date of first order for every branch
city = pd.read_csv('data_transform/city.csv')
common = pd.read_csv('data_original/common.csv')
common.Date = common.Date.apply(lambda x: pd.to_datetime(x))
groupped = common.groupby(['Point'])['Date'].min()
start_date = pd.DataFrame()
start_date['Point'] = groupped.index
start_date['FirstOrder'] = groupped.as_matrix()
city = pd.merge(city, start_date[['Point', 'FirstOrder']], how='left', on='Point')
city = city[city.columns.difference(['Unnamed: 0'])]
city.to_csv('data_transform/city.csv', index=None)
city.head()
'''

"\n# get date of first order for every branch\ncity = pd.read_csv('data_transform/city.csv')\ncommon = pd.read_csv('data_original/common.csv')\ncommon.Date = common.Date.apply(lambda x: pd.to_datetime(x))\ngroupped = common.groupby(['Point'])['Date'].min()\nstart_date = pd.DataFrame()\nstart_date['Point'] = groupped.index\nstart_date['FirstOrder'] = groupped.as_matrix()\ncity = pd.merge(city, start_date[['Point', 'FirstOrder']], how='left', on='Point')\ncity = city[city.columns.difference(['Unnamed: 0'])]\ncity.to_csv('data_transform/city.csv', index=None)\ncity.head()\n"

# internal_features.csv

In [41]:
src = pd.read_csv('data_transform/date.csv')
src[['Count', 'Point']] = pd.read_csv('data_original/common.csv')[['Count', 'Point']]
dst = pd.DataFrame() 

In [42]:
#tmp = pd.read_csv('data_transform/target.csv')
#tmp = pd.read_csv('data_transform/target.csv')[(tmp.IsTrain == True) | (tmp.IsValidation == True)]

#trainIndex = tmp.index

In [43]:
#src = pd.read_csv('data_transform/date.csv').iloc[trainIndex]
#src[['Count', 'Point']] = pd.read_csv('data_original/common.csv')[['Count', 'Point']].iloc[trainIndex]
#dst = pd.DataFrame() 

In [44]:
tmp = pd.read_csv('data_transform/target.csv')
validationIndex = tmp[tmp.IsValidation == True].index
src.loc[validationIndex, 'Count'] = None

In [45]:
src.head(10)

,Date,Year,Month,MonthDay,WeekDay,Point,YearDay,ThroughMonth,ThroughWeek,ThroughDay,Count
0,2011-10-06,2011,10,6,3,Сыктывкар-1,278,0,0,0,8.0
1,2011-10-07,2011,10,7,4,Сыктывкар-1,279,0,0,1,88.0
2,2011-10-09,2011,10,9,6,Сыктывкар-1,281,0,0,3,1.0
3,2011-11-15,2011,11,15,1,Сыктывкар-1,318,1,6,40,1.0
4,2011-12-08,2011,12,8,3,Сыктывкар-1,341,2,9,63,2.0
5,2011-12-09,2011,12,9,4,Сыктывкар-1,342,2,9,64,43.0
6,2011-12-10,2011,12,10,5,Сыктывкар-1,343,2,9,65,10.0
7,2011-12-11,2011,12,11,6,Сыктывкар-1,344,2,9,66,1.0
8,2011-12-13,2011,12,13,1,Сыктывкар-1,346,2,10,68,60.0
9,2011-12-14,2011,12,14,2,Сыктывкар-1,347,2,10,69,NaN


##  Next-Prev N Value

In [46]:
def GetValueForNPrevNextDay(n, target, src):
    targetCopy = target.copy()
    targetCopy.Count.loc[targetCopy.IsValidation == 1] = np.nan

    targetCopy['Point'] = src.Point

    city = src.Point.unique()

    result = np.array([np.nan] * targetCopy.shape[0])
    for i in city:
        value = targetCopy.Count[targetCopy.Point == i].shift(n)
        for j in value.index:
            result[j] = value[j]


    prefics = "Prev" if n > 0 else "Next"
    targetCopy[prefics + str(abs(n)) + "Day"] = pd.Series(result)
    del targetCopy["Point"]
    return targetCopy[prefics + str(abs(n)) + "Day"]

In [47]:
target["Prev1Day"] = GetValueForNPrevNextDay(1, target, src)
target["Prev2Day"] = GetValueForNPrevNextDay(2, target, src)
target["Prev3Day"] = GetValueForNPrevNextDay(3, target, src)
target["Prev7Day"] = GetValueForNPrevNextDay(7, target, src)

/home/laol/env/local/lib/python2.7/site-packages/pandas/core/indexing.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [48]:
target["Next1Day"] = GetValueForNPrevNextDay(-1, target, src)
target["Next1Day"] = GetValueForNPrevNextDay(-2, target, src)
target["Next1Day"] = GetValueForNPrevNextDay(-3, target, src)
target["Next1Day"] = GetValueForNPrevNextDay(-7, target, src)

In [49]:
target.to_csv('data_transform/target.csv', index = None)

In [50]:
target.head(15)

,Count,IsTest,IsTrain,IsValidation,Prev1Day,Prev2Day,Prev3Day,Prev7Day,Next1Day
0,8.0,0,1,0,NaN,NaN,NaN,NaN,1.0
1,88.0,0,1,0,8.0,NaN,NaN,NaN,60.0
2,1.0,0,1,0,88.0,8.0,NaN,NaN,NaN
3,1.0,0,1,0,1.0,88.0,8.0,NaN,96.0
4,2.0,0,1,0,1.0,1.0,88.0,NaN,134.0
5,43.0,0,1,0,2.0,1.0,1.0,NaN,NaN
6,10.0,0,1,0,43.0,2.0,1.0,NaN,100.0
7,1.0,0,1,0,10.0,43.0,2.0,8.0,NaN
8,60.0,0,1,0,1.0,10.0,43.0,88.0,73.0
9,NaN,1,0,0,60.0,1.0,10.0,1.0,NaN


##  Next-Prev N Existing Count
## НЕ ЗАПУСКАТЬ!!! - очень долго считается
## TODO: Оптимизировать

In [ ]:
date = pd.read_csv('data_transform/date.csv')
city = pd.read_csv('data_transform/city.csv')
target = pd.read_csv('data_transform/target.csv')

In [ ]:
tmp = pd.DataFrame()
tmp[["Count", "IsTest", "IsTrain", "IsValidation"]]= target[["Count", "IsTest", "IsTrain", "IsValidation"]]
tmp["Point"] = date.Point
tmp["Date"] = pd.to_datetime(date.Date)

In [ ]:
tmp.Count.loc[tmp.IsValidation == 1] = np.nan

In [ ]:
tmp.head()

In [ ]:
resultExistingMean = pd.DataFrame()

In [ ]:
def GetNextNExistingValue(n):
    result = np.array([np.nan] * tmp.shape[0])
    k = 1
    for i in range(tmp.shape[0]):
        currentDay = tmp.Date[i]
        targetCity = tmp.Point[i]
        tmpForCity = tmp[(tmp.Point == targetCity) &(tmp.IsTrain == 1)] #получение данных только по определенному 
        #городу и из Train части
        valueLessWhenN = (tmpForCity.Date.map(lambda x: (x  - currentDay).days) <= n) \
                        &(tmpForCity.Date.map(lambda x: (x  - currentDay).days) > 0)
        # получение городов, чья разница между первоночальной датой и вычисляемой меньше n и больше 0 (в днях)
        targetCount = tmpForCity.Count[valueLessWhenN] #получение Count переменных для этих городов
        targetCount = targetCount.dropna() #удаление отсутвующих значений для вычисления средних
        meanCount = np.mean(targetCount)
        result[i] = meanCount
        k += 1
        if ( (k % 500) == 1):
            print k / 500,
    return result

def GetPrevNExistingValue(n):
    result = np.array([np.nan] * tmp.shape[0])
    k = 1
    for i in range(tmp.shape[0]):
        currentDay = tmp.Date[i]
        targetCity = tmp.Point[i]
        tmpForCity = tmp[(tmp.Point == targetCity) &(tmp.IsTrain == 1)] #получение данных только по определенному 
        #городу и из Train части
        valueLessWhenN = (tmpForCity.Date.map(lambda x: (x  - currentDay).days) >= n) \
                        &(tmpForCity.Date.map(lambda x: (x  - currentDay).days) < 0)
        # получение городов, чья разница между первоночальной датой и вычисляемой меньше n и больше 0 (в днях)
        targetCount = tmpForCity.Count[valueLessWhenN] #получение Count переменных для этих городов
        targetCount = targetCount.dropna() #удаление отсутвующих значений для вычисления средних
        meanCount = np.mean(targetCount)
        result[i] = meanCount
        k += 1
        if ( (k % 500) == 1):
            print k / 500,
    return result

In [ ]:
resultExistingMean["NextExisting1Value"] = pd.Series(GetNextNExistingValue(1))

In [ ]:
resultExistingMean["NextExisting2Value"] = pd.Series(GetNextNExistingValue(2))

In [ ]:
resultExistingMean["NextExisting3Value"] = pd.Series(GetNextNExistingValue(3))

In [ ]:
resultExistingMean["NextExisting7Value"] = pd.Series(GetNextNExistingValue(7))

In [ ]:
resultExistingMean["NextExisting14Value"] = pd.Series(GetNextNExistingValue(14))

In [ ]:
resultExistingMean["NextExisting21Value"] = pd.Series(GetNextNExistingValue(21))

In [ ]:
resultExistingMean["NextExisting30Value"] = pd.Series(GetNextNExistingValue(30))

In [ ]:
resultExistingMean.to_csv('data_transform/resultExistingMean.csv', index=None)

In [ ]:
resultExistingMean["PrevExisting1Value"] = pd.Series(GetPrevNExistingValue(-1))

In [ ]:
resultExistingMean["PrevExisting2Value"] = pd.Series(GetPrevNExistingValue(-2))

In [ ]:
resultExistingMean["PrevExisting3Value"] = pd.Series(GetPrevNExistingValue(-3))

In [ ]:
resultExistingMean["PrevExisting7Value"] = pd.Series(GetPrevNExistingValue(-7))

In [ ]:
resultExistingMean["PrevExisting14Value"] = pd.Series(GetPrevNExistingValue(-14))

In [ ]:
resultExistingMean["PrevExisting21Value"] = pd.Series(GetPrevNExistingValue(-21))

In [ ]:
resultExistingMean["PrevExisting30Value"] = pd.Series(GetPrevNExistingValue(-30))

In [ ]:
resultExistingMean.to_csv('data_transform/resultExistingMean.csv', index=None)

## Mean

### by date independed from points

In [51]:
def calc_mean(src, group_column, target_column):
    mean = src.groupby([group_column])[target_column].mean()
    return src[group_column].apply(lambda x: mean[x])

def calc_median(src, group_column, target_column):
    median = src.groupby([group_column])[target_column].median()
    return src[group_column].apply(lambda x: median[x])

def calc_mode(src, group_column, target_column):
    mode = src.groupby([group_column])[target_column].mode().mean()
    return src[group_column].apply(lambda x: mode[x])

In [52]:
need_mean_list = src.columns.difference(['Date', 'Count', 'Point'])
for column in need_mean_list:
    dst['MeanFor' + column] = calc_mean(src, column, 'Count')
'''    
for column in need_mean_list:
    dst['MedianFor' + column] = calc_mean(src, column, 'Count')

for column in need_mean_list:
    dst['ModeFor' + column] = calc_mean(src, column, 'Count')
'''

"    \nfor column in need_mean_list:\n    dst['MedianFor' + column] = calc_mean(src, column, 'Count')\n\nfor column in need_mean_list:\n    dst['ModeFor' + column] = calc_mean(src, column, 'Count')\n"

In [53]:
dst.head(15)

,MeanForMonth,MeanForMonthDay,MeanForThroughDay,MeanForThroughMonth,MeanForThroughWeek,MeanForWeekDay,MeanForYear,MeanForYearDay
0,112.001349,105.840179,8.0,32.333333,32.333333,92.144521,81.761905,94.879032
1,112.001349,107.898594,88.0,32.333333,32.333333,123.380279,81.761905,91.006452
2,112.001349,105.792248,1.0,32.333333,32.333333,123.680556,81.761905,148.588235
3,102.960945,108.679322,1.0,1.000000,1.000000,87.408369,81.761905,127.435897
4,107.056321,109.635155,2.0,95.235294,14.000000,92.144521,81.761905,76.943820
5,107.056321,105.792248,43.0,95.235294,14.000000,123.380279,81.761905,77.311111
6,107.056321,101.936484,10.0,95.235294,14.000000,143.048428,81.761905,83.341772
7,107.056321,103.228209,1.0,95.235294,14.000000,123.680556,81.761905,104.192771
8,107.056321,100.536834,60.0,95.235294,97.500000,87.408369,81.761905,132.416667
9,107.056321,107.237121,NaN,95.235294,97.500000,87.944200,81.761905,119.550000


### by date depended from points

In [54]:
def map_group_two_key(group, x, y):
    return group[x][y]

def calc_mean_by_columns(src, group_columns, target_column):
    mean = src.groupby(group_columns)[target_column].mean()
    return src[group_columns].apply(lambda x: map_group_two_key(mean, *x), axis=1)



In [ ]:
need_list = src.columns.difference(['Date', 'Count', 'Point'])

In [ ]:
for column in need_list:
    dst['MeanFor' + column + 'ByPoints'] = calc_mean_by_columns(src, [column, 'Point'], 'Count')

In [ ]:

def calc_median_by_columns(src, group_columns, target_column):
    median = src.groupby(group_columns)[target_column].median()
    return src[group_columns].apply(lambda x: map_group_two_key(median, *x), axis=1)

def calc_moda_by_columns(src, group_columns, target_column):
    mode = src.groupby(group_columns)[target_column].aggregate(stats.mode)
    mode = mode.apply(lambda x: x[0][0])
    return src[group_columns].apply(lambda x: map_group_two_key(mode, *x), axis=1)



In [ ]:
'''
for column in need_list:
    dst['MedianFor' + column + 'ByPoints'] = calc_median_by_columns(src, [column, 'Point'], 'Count')

for column in need_list:
    dst['ModaFor' + column + 'ByPoints'] = calc_moda_by_columns(src, [column, 'Point'], 'Count')
'''

In [ ]:
dst["Point"] = pd.Series(common.Point.as_matrix())
dst.to_csv('data_transform/internal_features.csv', index=None)

# All in one

In [ ]:
data = pd.read_csv('data_transform/target.csv')
date = pd.read_csv('data_transform/date.csv')
city = pd.read_csv('data_transform/city.csv')
internal_features = pd.read_csv('data_transform/internal_features.csv')
dollar = pd.read_csv('data_transform/dollar.csv')

need_columns = date.columns
data[need_columns] = date[need_columns]

data = pd.merge(data, city, on='Point', how='left')

need_columns = internal_features.columns
data[need_columns] = internal_features[need_columns]

#data = pd.merge(data, dollar, on='Date', how='left')

data.to_csv('data_transform/all.csv', index=None)

In [ ]:
pd.isnull(data).sum() > 0

In [ ]:
data